Write some markdown to explain that this notebook will be mainly used for the capstone project.
Import the pandas library as pd.
Import the Numpy library as np.
Print the following the statement: Hello Capstone Project Course!

------------------------
This is the Markdown part
------------------------


In [3]:
#url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#from bs4 import BeautifulSoup
#import requests

#r=requests.get(url)
#content=r.content
#soup=BeautifulSoup(content,"html.parser")

In [ ]:
#print(soup)

In [ ]:
#wikitable=soup.find('table',class_='wikitable sortable')
#print(wikitable)
#now the information is stored in wikitable

In [9]:
#from pandas.io.json import json_normalize

#df = json_normalize(wikitable)

In [2]:
!pip install lxml
import pandas as pd # library for data analsysis

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)[0]
#dfs is a dataframe object

In [74]:
dfs.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [99]:
df=dfs[dfs.Borough!="Not assigned"]
df=df.reset_index(drop=True)
df.head(5)

(103, 3)

In [ ]:
for i in range(3):
    value=df.iloc[i]["Neighbourhood"]
    if value=="Not assigned":
        df.iloc[i]["Neighbourhood"]=df.iloc[i]["Borough"]
        print("find a NOT ASSIGNED")
    
df.head(5)

In [101]:
df.shape

(103, 3)

******************************************************************************************************************************************************
1. define a URL
2. read the html table into data frame
3. remove the rows of Borough="Not assigned" 
4. reset the index
5. change the value in Neighborhood="Not assigned" to the values of Borough

[The above code does not change anything as in this table, when the "Borough"="Not assigned" is removed, there is no "Neighborhood"="Not assigned"]
[But the code is made and tested to make sure the function is working]
******************************************************************************************************************************************************